In [142]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import os
#import shutil
import urllib.request

# Create Hawaii Fish Table

In [7]:
html = requests.get("https://www.fishbase.se/Country/CountryChecklist.php?showAll=yes&c_code=840B&vhabitat=all2&cpresence=present")
soup = bs(html.content, "html.parser")

In [115]:
try:
    fish_df = pd.read_csv("fish_df.csv")
except:
    print("Fish Dataframe not Found. Regenerating...")

    fish_df = pd.read_html(str(table))[0]

    # Add fish page Links
    links = []
    for anchor in table.findAll('a'):
        links.append("https://www.fishbase.se/Country/" + anchor["href"])
    fish_df["Link"] = links

    # Add fishbase ids
    fishbase_ids = [link.split('=')[-1] for link in links]
    fish_df.insert(0, "fishbase_id", fishbase_ids)

    # Add image counts and image page url for each species
    pic_links = ["https://www.fishbase.se/photos/thumbnailssummary.php?ID=" + fid for fid in fishbase_ids]  
    fish_df["image_page_link"] = pic_links
    
    image_counts = []
    for link in pic_links:
        page = requests.get(link)
        p_soup = bs(page.content, "html.parser")
        images = p_soup.findAll('img')
        image_counts.append(len(images))
    fish_df["image_counts"] = image_counts    

    fish_df.to_csv("fish_df.csv", index = False)

In [116]:
fish_df.head()

,fishbase_id,Order,Family,Species,Occurrence,FishBase name,Name,Link,image_counts,image_page_link
0,972,Beloniformes,Belonidae,Ablennes hians,native,Flat needlefish,'Aha'aha,https://www.fishbase.se/Country/CountrySpecies...,30,https://www.fishbase.se/photos/thumbnailssumma...
1,6652,Ovalentaria/misc,Pomacentridae,Abudefduf abdominalis,native,Green damselfish,Mamamo,https://www.fishbase.se/Country/CountrySpecies...,10,https://www.fishbase.se/photos/thumbnailssumma...
2,5689,Ovalentaria/misc,Pomacentridae,Abudefduf sordidus,native,Blackspot sergeant,Ao'aonui,https://www.fishbase.se/Country/CountrySpecies...,69,https://www.fishbase.se/photos/thumbnailssumma...
3,6630,Ovalentaria/misc,Pomacentridae,Abudefduf vaigiensis,native,Indo-Pacific sergeant,Indo-Pacific sergeant,https://www.fishbase.se/Country/CountrySpecies...,145,https://www.fishbase.se/photos/thumbnailssumma...
4,89,Scombriformes,Scombridae,Acanthocybium solandri,native,Wahoo,Ono,https://www.fishbase.se/Country/CountrySpecies...,111,https://www.fishbase.se/photos/thumbnailssumma...


# Download Images from Fishbase

In [120]:
if not os.path.exists("fishbase_images/"):
    os.makedirs("fishbase_images/")

In [146]:
for ind, row in fish_df.iterrows():
    id_path = "fishbase_images/" + str(row["fishbase_id"])
    if os.path.exists(id_path):
        continue
        
    os.makedirs(id_path)
        
    html = requests.get(row["image_page_link"])
    p_soup = bs(html.content, "html.parser")
    
    img_tags = p_soup.find_all('img')
    urls = []
    for url in [img['src'] for img in img_tags]:
        if (len(url.split('/'))) < 2 or url.split('/')[2] == "thumbnails" or url.split('.')[-1] not in set(["jpg", "gif", "png", "jpeg"]) :
            continue
        urls.append(url)
        
    base_url = "https://www.fishbase.se/"
    
    # Download Images
    for url in urls:
        full_url = base_url + url[3:]
        save_path = id_path+ "/" + url.split('/')[-1]
        urllib.request.urlretrieve(full_url, save_path)

['../images/species/abhia_u1.jpg', '../images/species/abhia_u2.jpg', '../images/species/abhia_m0.jpg', '../images/species/abhia_u0.jpg', '../images/species/abhia_j0.gif', '../images/species/abhia_j1.gif', '../images/species/abhia_u0.gif', '../tools/uploadphoto/uploads/img_3123.jpg', '../tools/uploadphoto/uploads/img_3134.jpg', '../tools/uploadphoto/uploads/1365915516_182.178.95.68.jpg', '../tools/uploadphoto/uploads/1564070934_188.114.103.151.jpg', '../images/species/abhia_j0.gif', '../images/species/abhia_j1.gif']


- Take curriculum learning like approach for google images? (train initial model without and then get confidences for google images)
- mask images with blue filters in augmentation